In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("/Users/kristenmaskery/Desktop/World_Weather_Analysis/Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Port-Cartier,CA,50.03,-66.87,55.99,47,40,6.93,scattered clouds
1,1,Krasnoselkup,RU,65.70,82.47,32.36,94,100,24.00,overcast clouds
2,2,Uvalde,US,29.37,-99.77,81.00,61,40,6.93,scattered clouds
3,3,Arcata,US,40.87,-124.08,53.60,100,90,2.89,fog
4,4,Saint-Leu,RE,-21.15,55.28,73.40,83,75,13.87,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp=float(input("What is the minimum temperature you would like for your vacation? "))
max_temp=float(input("What is the maximum temperature you would like for your vacation?"))

What is the minimum temperature you would like for your vacation? 70
What is the maximum temperature you would like for your vacation?90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_temp_df=city_data_df.loc[(city_data_df["Max Temp"]<= max_temp) &\
                                   (city_data_df["Max Temp"] >= min_temp)]
preferred_temp_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Uvalde,US,29.37,-99.77,81.00,61,40,6.93,scattered clouds
4,4,Saint-Leu,RE,-21.15,55.28,73.40,83,75,13.87,broken clouds
5,5,Hambantota,LK,6.12,81.12,80.51,77,93,16.73,overcast clouds
6,6,Ribeira Grande,PT,38.52,-28.70,75.20,64,40,17.22,scattered clouds
12,12,Castro,BR,-24.79,-50.01,78.96,55,58,10.94,broken clouds


In [6]:
# 4a. Determine if there are any empty rows.
preferred_temp_df.count()

City_ID                289
City                   289
Country                288
Lat                    289
Lng                    289
Max Temp               289
Humidity               289
Cloudiness             289
Wind Speed             289
Current Description    289
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_preferred_temp_df=preferred_temp_df.dropna()
clean_preferred_temp_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Uvalde,US,29.37,-99.77,81.00,61,40,6.93,scattered clouds
4,4,Saint-Leu,RE,-21.15,55.28,73.40,83,75,13.87,broken clouds
5,5,Hambantota,LK,6.12,81.12,80.51,77,93,16.73,overcast clouds
6,6,Ribeira Grande,PT,38.52,-28.70,75.20,64,40,17.22,scattered clouds
12,12,Castro,BR,-24.79,-50.01,78.96,55,58,10.94,broken clouds
...,...,...,...,...,...,...,...,...,...,...
696,696,Jiazi,CN,22.88,116.07,76.89,81,65,7.11,light rain
697,697,Kalianget,ID,-7.05,113.93,82.35,73,73,10.16,broken clouds
698,698,Gasa,PS,31.50,34.47,78.80,57,20,8.05,few clouds
700,700,Hit,IQ,33.64,42.83,83.50,23,0,6.51,clear sky


In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_preferred_temp_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Uvalde,US,81.00,scattered clouds,29.37,-99.77,
4,Saint-Leu,RE,73.40,broken clouds,-21.15,55.28,
5,Hambantota,LK,80.51,overcast clouds,6.12,81.12,
6,Ribeira Grande,PT,75.20,scattered clouds,38.52,-28.70,
12,Castro,BR,78.96,broken clouds,-24.79,-50.01,
15,Aklera,IN,78.06,clear sky,24.42,76.57,
17,Mangochi,MW,86.32,scattered clouds,-14.48,35.26,
18,Luanda,AO,73.22,clear sky,-8.84,13.23,
21,Buala,SB,79.59,few clouds,-8.14,159.59,
27,Rikitea,PF,72.72,overcast clouds,-23.12,-134.97,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key}

# 6b. Iterate through the hotel DataFrame.
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
hotel_df

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Uvalde,US,81.00,scattered clouds,29.37,-99.77,
4,Saint-Leu,RE,73.40,broken clouds,-21.15,55.28,La Caz Oceane
5,Hambantota,LK,80.51,overcast clouds,6.12,81.12,Bungalow 63
6,Ribeira Grande,PT,75.20,scattered clouds,38.52,-28.70,Quinta da Meia Eira
12,Castro,BR,78.96,broken clouds,-24.79,-50.01,CHACARA BAILLY
...,...,...,...,...,...,...,...
696,Jiazi,CN,76.89,light rain,22.88,116.07,Hotel Jin Hao
697,Kalianget,ID,82.35,broken clouds,-7.05,113.93,Pesanggrahan Garam
698,Gasa,PS,78.80,few clouds,31.50,34.47,رابطة الفنانين الفلسطينيين
700,Hit,IQ,83.50,clear sky,33.64,42.83,Aboody Alhachem


In [11]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Uvalde,US,81.00,scattered clouds,29.37,-99.77,
4,Saint-Leu,RE,73.40,broken clouds,-21.15,55.28,La Caz Oceane
5,Hambantota,LK,80.51,overcast clouds,6.12,81.12,Bungalow 63
6,Ribeira Grande,PT,75.20,scattered clouds,38.52,-28.70,Quinta da Meia Eira
12,Castro,BR,78.96,broken clouds,-24.79,-50.01,CHACARA BAILLY
...,...,...,...,...,...,...,...
696,Jiazi,CN,76.89,light rain,22.88,116.07,Hotel Jin Hao
697,Kalianget,ID,82.35,broken clouds,-7.05,113.93,Pesanggrahan Garam
698,Gasa,PS,78.80,few clouds,31.50,34.47,رابطة الفنانين الفلسطينيين
700,Hit,IQ,83.50,clear sky,33.64,42.83,Aboody Alhachem


In [ ]:
# 8a. Create the output File (CSV)
output_data_file=
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [ ]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [ ]:
# 11a. Add a marker layer for each city to the map. 

# 11b. Display the figure
